In [1]:
# KNN + Boarder Detection + Normal Direction Constrains

# 1) KNN search: several candidates of corresponding points
# 2) Boarder Detection + Normal Direction constrains => remove some bad correspondences

In [2]:
import math
import pickle
import os
import time
import scipy
import numpy as np
from scipy.optimize import minimize
from sklearn.neighbors import NearestNeighbors


## SMPL model
data_root   = '/home/yan/Data2/NOMO_Project_Summary/'
root_folder = data_root + 'MVA/MVA_rebuttal_code/Shape_Parameter_Optimization/'
import sys
sys.path.append(root_folder+'SMPL/SMPL_python_v.1.0.0/smpl/')
from smpl_webuser.serialization import load_model

smpl_model  = root_folder + 'SMPL/SMPL_python_v.1.0.0/smpl/models/basicModel_m_lbs_10_207_0_v1.0.0.pkl'
scan_folder =  data_root + 'Data/TC2_scan/ori_scan/re_name/dataset01/male/'
out_folder  = data_root + 'MVA/MVA_rebuttal_code/Shape_Parameter_Optimization/optimized_mesh/dataset01/'

# pre-process initial params
param_folder = '/home/yan/Data2/3D_Body_Reconstruction/Dataset/scans/Optimized_Registered_NOMO3D_Dataset1/Original_A_Posed/parameter_male/'

# load SMPL model
model = load_model(smpl_model)
scan_list = os.listdir(scan_folder)

deg2rad = 1.0 / 180 * math.pi
bnds = ((-20*deg2rad, -5*deg2rad), (-5*deg2rad,  5*deg2rad), (-90*deg2rad, -30*deg2rad), 
        (-20*deg2rad, -5*deg2rad), (-5*deg2rad,  5*deg2rad), (30*deg2rad,   90*deg2rad),
        (5*deg2rad,   30*deg2rad), (-30*deg2rad,-5*deg2rad), (0*deg2rad,    10*deg2rad),  
        (5*deg2rad,   30*deg2rad), (5*deg2rad,  30*deg2rad), (-10*deg2rad,   0*deg2rad),  
        (-5*deg2rad,   5*deg2rad), (-5*deg2rad,  5*deg2rad), (0*deg2rad,    15*deg2rad), 
        (-5*deg2rad,   5*deg2rad), (-5*deg2rad,  5*deg2rad), (-15*deg2rad,   0*deg2rad),
        (0*deg2rad,   10*deg2rad), (0*deg2rad,  10*deg2rad), (-5*deg2rad,    5*deg2rad),
        (-10*deg2rad,  0*deg2rad), (-10*deg2rad, 0*deg2rad), (-10*deg2rad,   0*deg2rad), 
        (-10*deg2rad,  0*deg2rad), (0*deg2rad,  10*deg2rad), (0*deg2rad,    10*deg2rad),
        (-3, 3), (-3, 3), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5))

In [3]:
def loadTarget(scanFile):
    ''' load NOMO3D scans, return Vertices and Faces'''
    with open(scanFile, 'r') as fp:
        content = fp.readlines()
    content = [x.strip() for x in content]

    targetV = []
    targetF = []
    for line in content:
        CC = line.split()
        if len(CC) > 0:
            if CC[0] == 'v':
                targetV.append([float(CC[1]), float(CC[2]), float(CC[3])])
            elif CC[0] == 'f':
                targetF.append([int(CC[1]), int(CC[2]), int(CC[3])])

    targetV = np.array(targetV)
    targetV = targetV - np.mean(targetV, 0)
    targetF = np.array(targetF, dtype=int) - 1 # .obj file, is 0-based, started from 0
    
    return targetV, targetF

def loadInitParam(paramFile):
    '''load initial parameters, including shape and pose from the previous experiments'''
    if paramFile == None:
        init_betas = np.zeros(10)
        init_pose = np.zeros(72)
        deg2rad = 1.0 / 180 * math.pi
        init_pose[48:60] = np.multiply(deg2rad, [-12, 2, -69, -12,-2, 69, 16, -18, 4, 16, 18, -4])
        init_pose[3:18] = np.multiply(deg2rad, [1, 0, 9, 1, 0, -8, 4, 4, 0, -3, -2, -2, -3, 2, 2])
    else:
        with open(paramFile, 'rb') as f:
            param_data = pickle.load(f)
        init_pose = param_data[:72]
        init_betas = param_data[72:]
    
    return init_pose, init_betas

def normalize_v3(arr):
    '''Normalize a numpy array of 3 component vectors shape=(n, 3)'''
    lens = np.sqrt(arr[:, 0]**2 + arr[:, 1]**2 + arr[:, 2]**2)
    arr[:, 0] /= lens
    arr[:, 1] /= lens
    arr[:, 2] /= lens
    return arr

def getNormals(V, F):
    ''' Creating a normal per triangle, is done by the cross product of two vectors on this triangle.
    Creating smooth normals, i.e. one normal per vertex is done 
    by averaging the normal of all the triangles one vertex is part of.
    
    https://sites.google.com/site/dlampetest/python/calculating-normals-of-a-triangle-mesh-using-numpy'''
    
    norm = np.zeros(V.shape, dtype=V.dtype)
    # Create an indexed view into the vertex array using the array of three indices for triangles
    tris = V[F] 
    # Calculate the normal for all the triangles, 
    # by taking the cross product of the vectors v1-v0, and v2-v0 in each triangle 
    n = np.cross( tris[::,1 ] - tris[::,0]  , tris[::,2 ] - tris[::,0] )
    n = normalize_v3(n)
    # Multiple triangles would then contribute to every vertex
    norm[F[:, 0]] += n
    norm[F[:, 1]] += n
    norm[F[:, 2]] += n
    norm = normalize_v3(norm)
    
    return norm

def getBoarderV(Faces):
    '''the boarder of the mesh are those edges which belong to only one polygon'''
    boundV = []
    edges01 = np.array([Faces[:, 0], Faces[:, 1]], dtype=int).T
    edges02 = np.array([Faces[:, 0], Faces[:, 2]], dtype=int).T
    edges03 = np.array([Faces[:, 1], Faces[:, 2]], dtype=int).T
    edge = np.sort(np.concatenate((edges01, edges02, edges03), axis=0))
    
    unique_e, n_counts = np.unique(edge, axis=0, return_counts=True)
    boarder_edge = unique_e[n_counts==1, :]
    boundV = np.unique(boarder_edge.flatten())
    
    return boundV

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'"""
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.absolute(np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)))

In [4]:
def meshDist(param):
    '''the distance between the SMPL model and the target scan'''
    
    model.betas[:] = param[27:]
    model.pose[48:60] = param[:12]
    model.pose[3:18] = param[12:27]
    
    modelF = model.f # 0-based, started from 0
    modelV = model.r - np.mean(model.r, 0)
    modelNorm = getNormals(modelV, modelF)
    
    # KNN correspondence search, find 5 nearest neighbour as the estimated correspondences
    N_neigh = 3
    _, t_idx = kdtree.query(modelV, k=N_neigh)
    
    # the correspondence should : 
    #      1) the similar normal, the angle is small
    #      2) the small distance,
    #      3) not at the boarder,
    
    W = np.ones((6890, 1), dtype=float) # 0 for bad correspondences, otherwise 1
    correspondV = np.zeros((6890, 3), dtype=float)
    
    dist = 0
    for idx in range(6890):
        m_normal = modelNorm[idx, :]
        good_neigh_idx = 0
        minAngle = 100
        for c_id in range(N_neigh):
            neigh_idx = t_idx[idx, c_id]
            t_normal = targetNorm[neigh_idx, :]
            angle = angle_between(t_normal, m_normal)
            if angle <= minAngle:
                minAngle = angle
                correspondV[idx, :] = targetV[neigh_idx, :]
                good_neigh_idx = neigh_idx
                
        dist_v = np.sqrt(np.sum(np.square(modelV[idx, :] - targetV[good_neigh_idx, :])))
        if dist_v > 0.1:           # 10 cm
            W[idx, :] = 0
        if minAngle > math.pi / 4: # 45 degree
            W[idx, :] = 0
        if good_neigh_idx in boarderV:
            W[idx, :] = 0
        
        dist = dist + dist_v * W[idx, :]
    # weighted mean distance between the model and the correspondences vertices
#     dist = np.sum(np.multiply(W, np.sqrt(np.sum(np.square(modelV - correspondV), axis=1)))) / np.count_nonzero(W)
    dist = dist / np.count_nonzero(W)
    
    return dist * 1000 # [mm]

In [5]:
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)


# start_time = time.time()

for scan in scan_list:
    scan_name = scan[:-4]

    targetV, targetF = loadTarget(scan_folder+scan_name+'.obj') # registered mesh, (6890, 3)
    boarderV = getBoarderV(targetF)                             # the vertices on the boarder edges
    targetNorm = getNormals(targetV, targetF)                   # the normal for every target vertices
    kdtree = scipy.spatial.cKDTree(targetV, leafsize=6890)      # KNN search
    
    paramFile = param_folder + scan_name + '.pkl'               # None for female samples                       
    init_pose, init_betas = loadInitParam(paramFile)            # female does not have the pre-param
    
    init_param        = np.zeros(37)                            # 27 part pose parameters + 10 shape param
    init_param[:12]   = init_pose[48:60]
    init_param[12:27] = init_pose[3:18]
    init_param[27:]   = init_betas
    
    start_time = time.time()
    res = minimize(meshDist, init_param, method='L-BFGS-B', bounds=bnds, options={'gtol': 1e-6, 'disp': True})
    end_time = time.time()
    print "eclipse time [second] : ", end_time - start_time
    
    optimal_param = res.x
    print scan_name, " : ", optimal_param[27:]
    
    # save
    model.betas[:]    = optimal_param[27:]
    model.pose[48:60] = optimal_param[:12]
    model.pose[3:18]  = optimal_param[12:27]

    with open(out_folder + 'scan2Optimized_shape_pose/male/' + scan_name + '.obj', 'w') as fp:
        for v in model.r: # [m]
            fp.write( 'v %f %f %f\n' % ( v[0], v[1], v[2]) )
        for f in model.f+1: # Faces are 1-based, not 0-based in obj files
            fp.write( 'f %d %d %d\n' %  (f[0], f[1], f[2]) )

    optimal_param = np.zeros(82)
    optimal_param[72:]   = res.x[27:]
    optimal_param[48:60] = res.x[:12]
    optimal_param[3:18]  = res.x[12:27]

    with open(out_folder + 'scan2Optimized_shape_pose/male_param/' + scan_name + '.pkl', 'w') as fp:
        pickle.dump({'param_data': optimal_param}, fp)

# end_time = time.time()

# print "eclipse time [second] : ", end_time - start_time

eclipse time [second] :  3440.24106717
male_0112  :  [ 0.55902957 -0.86537932  0.63162387 -0.08697901  0.37198876  0.27055622
 -0.17170351  0.03196573 -0.02116054  0.33517577]
eclipse time [second] :  5037.69454312
male_0015  :  [ 0.14747469  1.40732994  0.22942437 -0.32754073  0.11215355 -0.36446125
  0.03366784 -0.05695375 -0.01140323  0.28870494]
eclipse time [second] :  12976.6260431
male_0074  :  [-2.63648117  2.39604864 -0.68944559 -0.59524442 -0.38607633 -0.48625721
  0.63456898  0.08294885  0.02408454  0.07148474]
eclipse time [second] :  11411.8426499
male_0110  :  [-1.18099682  0.54785142  0.17810701 -0.44480075  0.22493666 -0.16135014
  0.07075329  0.1900377  -0.01156646  0.18806457]
eclipse time [second] :  2748.97217798
male_0064  :  [ 0.90895703 -0.30741404  1.34867793 -0.06202161  0.30446008 -0.02899133
  0.15016395  0.01234877  0.02573058  0.53037847]
eclipse time [second] :  2534.25702
male_0140  :  [-1.64991957  0.14113296  0.93171634 -0.50623832  0.32016888  0.020840

eclipse time [second] :  5968.14207006
male_0083  :  [-0.31495692  0.05305971 -0.01205434 -0.79600332  0.12488378 -0.01712656
  0.31796778  0.30264955 -0.12203476  0.30865958]
eclipse time [second] :  11274.2477789
male_0005  :  [-3.          0.38739444 -0.55353336 -0.12595172 -0.40319941 -0.52210166
  1.09625275  0.73678196 -0.14261103  0.09788453]
eclipse time [second] :  3645.10602498
male_0169  :  [ 0.92149066  0.32865888  0.02804299 -0.1170915   0.53388426  0.0424022
  0.17914146  0.17740122  0.00583438  0.22398822]
eclipse time [second] :  6353.95057988
male_0176  :  [-3.          2.12189711 -0.43304263  0.12808039 -0.19718643 -0.3626403
  0.03434743  0.30750373 -0.13310574  0.35473193]
eclipse time [second] :  3734.23530197
male_0051  :  [-0.34166593  0.46603869  0.73760541 -0.18785976  0.40068175  0.01987149
 -0.10458254 -0.06977405  0.02294415  0.2612446 ]
eclipse time [second] :  4251.6104691
male_0073  :  [ 1.76585665 -0.1279488  -0.1119705  -0.29681839  0.01152742 -0.249966

eclipse time [second] :  4786.23816085
male_0120  :  [ 0.32283511 -0.05597573  0.44103307 -0.21024197  0.36456142 -0.02915475
  0.04130037  0.14951315 -0.00651369  0.52273122]
eclipse time [second] :  8550.58372402
male_0060  :  [-3.          2.59730612 -0.22231617 -0.70556555 -0.58724724 -1.13855816
  0.88759284  0.40508812  0.06303289 -0.0243543 ]
eclipse time [second] :  7787.04923415
male_0031  :  [-3.          2.11121612 -0.97228427 -0.68741674 -1.44072837 -1.30954884
  0.42417083  0.72388951 -0.04558464 -0.07488601]
eclipse time [second] :  5424.24607396
male_0157  :  [ 0.55740641  0.07721006  0.23449278  0.28085226 -0.10041589 -0.12298426
 -0.01701468 -0.02001698  0.05304858  0.07981611]
eclipse time [second] :  12538.8067892
male_0108  :  [-3.          1.66031035 -1.78945003 -1.35879222  0.36036521 -1.57198142
  1.1841433   1.48956783 -0.70668178  0.73860747]
eclipse time [second] :  11263.3925829
male_0179  :  [-3.          0.31918992  0.25676212 -0.11094217  0.05166987  0.123

eclipse time [second] :  4209.98800588
male_0084  :  [ 0.13185868 -0.26391422  0.33065897 -0.13244487  0.09376184  0.08701443
 -0.06131255  0.0562914  -0.06120648  0.17480393]
eclipse time [second] :  2590.36390018
male_0142  :  [-0.33562113 -0.90661954  0.0515412  -0.66046381  0.03386092  0.06984949
 -0.00268307  0.0509649  -0.08119509  0.34905558]
eclipse time [second] :  13522.9615972
male_0177  :  [-3.          2.08138645 -1.0406834  -1.38731451 -0.18926694 -3.61409013
 -0.38856954  1.92256411 -0.13638254  0.05192027]
eclipse time [second] :  4204.08977604
male_0089  :  [-0.56179159  1.0991815   0.19832943 -0.19338465 -0.43279996 -0.1324273
  0.05795149  0.08149358 -0.07769542  0.09364421]
eclipse time [second] :  4982.49387407
male_0052  :  [-0.90774634  1.84414857 -0.55728328  0.18103463  0.04251337 -0.25644066
  0.26732547  0.10643716  0.13792832 -0.13950929]
eclipse time [second] :  10348.9388461
male_0091  :  [ 1.3256923   0.44865347 -0.28033482 -0.16201194  0.10307736 -0.3172

In [6]:
print "done"

done
